# Selección de características en PySpark

El propósito de este notebook es demonstrar la mayoría de las técnicas de selección de características y reducción de dimensiones con los módulos del Spark Machine Learning.

## 1. Fundamentos de selección de características y reducción de la dimensionalidad

La selección de características es el proceso de reducir el número de variables de entrada cuando desarrollamos un modelo de aprendizaje automático. El objetivo del proceso es eliminar aquellas variables que sea **redundantes** o **irrelevantes** al problema.

Es deseable reducir el número de variables de entrada para reducir el costo computacional, quitar el ruido de los datos y lograr un modelo más sencillo con performance similar o superior.

Por lo general, se utiliza métodos estadísticos que involucran evaluar las relaciones entre las variables de entrada y la variable de salida, y seleccionando las variables que presentan una correlación superior. Algunos de beneficios de utilizar una técnica para seleccionar las variables incluyen:
* Simplificación de los modelos para que sean mejor interpretados,
* Tiempos de entrenamiento más cortos, y
* Mejora en la compartibilidad de los datos con el modelo de aprendizaje automático.

Otras técnicas que logran el mismo propósito de la selección de caracteristicas se agrupan como técnicas de **reducción de la dimensionalidad**. A diferencia de las técnicas de selección de características, la reducción de dimensiones consiste en aplicar enfoques lineales y no lineales para transformar los datos y reducir la dimensionalidad de los mismos. En otras palabras, los datos son alterados para que sean representados en un espacio de menores dimensiones.

### 1.1 Los desafíos de seleccionar las variables
Con la popularidad de la Big Data, no sería poco común enfrentarnos a los desafíos de trabajar con conjuntos de datos de muchas dimensiones. Los desafíos pueden ser resumidas en dos:
* Comprensión de las estructuras internas de cada variable y
* Escalabilidad y estabilidad computacional.

El proceso de reducir la cantidad de variables de entrada **representa pérdida de alguna información** en variadas proporciones, lo que afectaría la performance de cualquier modelo de machine learning.

Por otro lado, trabajar con minería de datos de muchas dimensiones puede llevar a un fenómeno conocido como la **maldición de dimensionalidad**. Dicho fenómeno aparece cuando los datos se vuelven más escasos en un espacio de grandes dimensiones, afectando negativamente a los algoritmos de aprendizaje automático, que están diseñados para trabajar con datos de pocas dimensiones.

El practicante de aprendizaje automático debe ser capaz de seleccionar las dimensiones que sean relevantes para el modelo, sin que esto represente pérdida significativa de la información o degradación negativa del rendimiento del aprendizaje. Esto implica tener un conocimiento sobre las diferentes estrategias para exitosamente llevar a cabo este proceso.


### 1.2. Las estrategias de selección de características

Los métodos o algoritmos de selección de características pueden ser dividos en:
* Métodos de envoltura,
* Métodos de filtro, y
* Métodos incrustados o implícitos.

Los **métodos de envoltura** utilizan la performance de un modelo entrenado o preentrenado para evaluar la relevancia de cada característica. Básicamente, se elige subconjuntos de variables para crear una serie de modelos. Las variables que resultan en un modelo con la mejor performance son seleccionados. Estos métodos suelen ser muy lentos y poco utilizados en la práctica, dado que escanea el conjunto de datos en varias ocasiones.

Algunos ejemplos incluyen **Eliminación Recursiva de Características (RFE)** y los algoritmos genéticos **(AG)**.
![Agrupados](assets/4.png)

Los **métodos de filtro** utilizan técnicas estadísticas para evaluar la relación entre cada variable de entrada y la variable de salida. Las variables de entradas que están mejor relacionadas con la variable de salida son seleccionadas.

Algunos ejemplos de técnicas estadísticas incluyen **chi-cuadrado**, **coeficiente de Pearson** y **coeficiente de ANOVA**.
![Agrupados](assets/5.png)

Los **métodos incrustados o implícitos** son los métodos que están incorporados en los algoritmos de aprendizaje automático.

Algunos ejemplos incluyen:

* **Regresión con regularización L1(Lasso) y L2(Ridge)**: tienen funciones de penalización para reducir el sobreajuste del modelo.
* **Árboles de decisión**: através de la recursión se lograr reducir la cantidad de variables de entrada hasta un cierto criterio.

![Agrupados](assets/6.png)

### 1.3. Selección de características y reducción de dimensionalidad en Apache Spark

En este notebook, nos enfocaremos en los **métodos de filtro** para selección las variables en base a pruebas estadísticas, además de utilizar los módulos disponibles para reducir la dimensionalidad de un conjunto de datos con variables de entrada homogéneas.

La librería de Spark ML viene con algunos módulos para reducir estadísticamente la cantidad de variables en un conjunto de datos. Estos se resumen de la siguiente manera:
* **Selección de características**
    * Remover características con poca correlación con la variable de salida: UnivariateFeatureSelection
    * Remover características con poca varianza: VarianceThresholdSelector
    
   
* **Reducción de la dimensionalidad**
    * Principal Component Analysis (PCA)
    
En los siguientes ejemplos, veremos cómo se aplican cada una de estas técnicas.

In [6]:
!pwd

/mnt/main/home/kauvin/Git/Spark-StudyClub/Spark - The Definite Guide/session_18


In [0]:
# Filtrar los warning
import warnings
warnings.filterwarnings("ignore")

# Importar las librerías
import findspark
findspark.init('/usr/local/spark') "
import pyspark.sql.functions as f
import pyspark.sql.types as t
import pandas as pd

# Iniciar SparkSession
spark = SparkSession \
        .builder \
        .getOrCreate()

## 2. Ejemplo de selección de características - Predicción de la popularidad de las noticias

El conjunto de datos contiene los datos sobre los artículos publicados en Mashable en un periodo de dos años. El objetivo es predecir la cantidad de compartidos en las redes sociales (popularidad).

In [0]:
# Leer los datos
df = spark\
     .read\
     .option("header", "true")\
     .option("inferSchema", True)\
     .csv("/FileStore/tables/OnlineNewsPopularity.csv")\

spark.sparkContext.setLogLevel("ERROR")

# Reducir el número de Shuffle partitions para 5
spark.conf.set("spark.sql.shuffle.partitions", "200")

# No truncar las columnas
pd.set_option("display.max_columns", None)

# Imprimir el shape
print(f"Este dataset tiene {len(df.columns)} columnas y {df.count()} observaciones")

# Ver los primeros registros
df.limit(5).toPandas()

Este dataset tiene 61 columnas y 39644 observaciones


,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,4.680365,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,496.0,496.0,496.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500331,0.378279,0.040005,0.041263,0.040123,0.521617,0.092562,0.045662,0.013699,0.769231,0.230769,0.378636,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593.0
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,0.0,4.913725,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.799756,0.050047,0.050096,0.050101,0.050001,0.341246,0.148948,0.043137,0.015686,0.733333,0.266667,0.286915,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711.0
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,0.0,4.393365,6.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,918.0,918.0,918.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.217792,0.033334,0.033351,0.033334,0.682188,0.702222,0.323333,0.056872,0.009479,0.857143,0.142857,0.495833,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500.0
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,0.0,4.404896,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028573,0.419300,0.494651,0.028905,0.028572,0.429850,0.100705,0.041431,0.020716,0.666667,0.333333,0.385965,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200.0
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,0.0,4.682836,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,545.0,16000.0,3151.157895,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028633,0.028794,0.028575,0.028572,0.885427,0.513502,0.281003,0.074627,0.012127,0.860215,0.139785,0.411127,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505.0


### 2.1. Exploración de los datos

Luego de analizar cuidadosamente los datos de entrada, no se detectó la existencia de registros que contenían valores nulos o faltantes:

In [0]:
# Buscar por valores nulos
sum_nulls = df.select([f.count(f.when(f.isnan(c) | f.col(c).isNull(), c)).alias(c) for c in df.columns]
   ).limit(5).toPandas().sum(axis=1)[0]
if sum_nulls == 0:
    print("No detectaron valores nulos")
else:
    print(f"La cantidad total de valores nulos es de {sum_nulls}")

No detectaron valores nulos


### 2.2. Eliminación de variables irrelevantes

El primero paso en la selección de caracteristicas sería la identificación y eliminación de las variables que a simple vista no son relevantes para el analisis. Estas variables serían la `url` (que representa la URL de la publicación en la revista) y el `timedelta` (que representa los días entre la publicación y la generación del conjunto de datos).

In [0]:
# Eliminar las columnas relevantes
df = df.drop("url", "timedelta")

### 2.3. Transformación mediante VectorAssembler

In [0]:
from pyspark.ml.feature import VectorAssembler

# Definir las variables de entrada y de salida
inputcols = df.drop(" shares").columns
outputcol = "features"

# Aplicar el VectorAssembler
assembler = VectorAssembler(inputCols = inputcols, outputCol=outputcol)
features = assembler.transform(df)

# Eliminar las otras columnas
for column in inputcols:
    features = features.drop(column)
    
# Renombrar la variable de salida
features = features.withColumnRenamed(" shares", "label")
    
# Visualizar el resultado
features.limit(5).toPandas()

,label,features
0,593.0,"(731.0, 12.0, 219.0, 0.663594466988, 0.9999999..."
1,711.0,"(731.0, 9.0, 255.0, 0.604743080614, 0.99999999..."
2,1500.0,"(731.0, 9.0, 211.0, 0.575129530699, 0.99999999..."
3,1200.0,"(731.0, 9.0, 531.0, 0.503787877834, 0.99999999..."
4,505.0,"(731.0, 13.0, 1072.0, 0.41564561695, 0.9999999..."


### 2.4. Selección de características mediante el UnivariateFeatureSelector

El UnivariateFeatureSelector es un módulo robusto para seleccionar las variables de entradas en Spark ML. Se aplica para las variables de entrada y de salida continuas o categóricas.

Se aplica una función de puntaje basada en una prueba estadística univariada para seleccionar las top **n** variables del conjunto de datos. El valor de **n** puede ser determinado por los métodos: 
* `numTopFeatures`: se elige un determinado número de variables.
* `percentile`: es similar la numTopFeatures pero se elige una fracción de las variables.
* `fpr`: se elige las variables que están por debajo de un umbral de valor p.
* `fdr`: se elige las variables utilizando el procedimiento de Benjamini-Hochberg cuya tasa de falso descubrimiento sea menor al especificado
* `fwe`: se elige todas las variables cuyos valores p están por debajo de un umbral. El umbral se escala por 1/numFeatures, controlando así la tasa de error de selección por familia.

La función de puntaje es elegida en base al tipo de las variables de entrada y de la variable de salida de la siguiente manera:

| Tipo (variables de entrada) | Tipo (variable de salida) | Función de puntaje     |
|-----------------------------|---------------------------|------------------------|
| Categórica                  | Categórica                | chi-cuadrado (chi2)    |
| Continua                    | Categórica                | ANOVATest (f_classif)  |
| Continua                    | Continua                  | F-value (f_regression) |

La función de **chi-cuadrado** se basa en la prueba χ² de independencia (se usa esta función en lugar de utilizar el  módulo **ChiSqSelector** que está descontinuado). El **ANOVATest** se basa en la prueba de análisis de varianza (ANOVA) para determinar el valor de F entre las variables de entrada y de salida. El **F-value** aplica pruebas de regresión lineal univariante que devuelven el estadístico F y los valores de p.

#### 2.4.1. Ejemplo de UnivariateFeatureSelector con numTopFeatures

In [0]:
from pyspark.ml.feature import UnivariateFeatureSelector

# Inicializar el UnivariateFeatureSelector
selector = UnivariateFeatureSelector(featuresCol="features", outputCol="selectedFeatures",
                                     labelCol="label", selectionMode="numTopFeatures")

selector.setFeatureType("continuous").setLabelType("continuous").setSelectionThreshold(30)

# Aplicar el UnivariateFeatureSelector
result = selector.fit(features).transform(features)

# Validar la transformación
prev_num_feat = result.schema["features"].metadata["ml_attr"]["num_attrs"]
new_num_feat = result.schema["selectedFeatures"].metadata["ml_attr"]["num_attrs"]
print(f"El número total de variables de entrada pasó a ser de {prev_num_feat} a {new_num_feat}")

El número total de variables de entrada pasó a ser de 59 a 30


#### 2.4.2. Ejemplo de UnivariateFeatureSelector con fpr

In [0]:
# Inicializar el UnivariateFeatureSelector
selector = UnivariateFeatureSelector(featuresCol="features", outputCol="selectedFeatures",
                                     labelCol="label", selectionMode="fpr")

selector.setFeatureType("continuous").setLabelType("continuous").setSelectionThreshold(0.05)

# Aplicar el UnivariateFeatureSelector
result = selector.fit(features).transform(features)

# Validar la transformación
prev_num_feat = result.schema["features"].metadata["ml_attr"]["num_attrs"]
new_num_feat = result.schema["selectedFeatures"].metadata["ml_attr"]["num_attrs"]
print(f"El número total de variables de entrada pasó a ser de {prev_num_feat} a {new_num_feat}")

El número total de variables de entrada pasó a ser de 59 a 34


### 2.5. Selección de características mediante el VarianceThresholdSelector

El VarianceThresholdSelector es el módulo que remueve las variables con poca varianza. Se define un `varianceThreshold` para especificar el nivel de varianza mínimo para seleccionar un variable. Puede ser utilizado para problemas de aprendizaje supervisado y no supervisado.

Se trata de un enfoque sencillo para seleccionar las variables. Por defecto, remueve todas las variables que tengan cero de varianza (valor constante).

Es importante asegurarnos primeramente de que las variables estén en una misma escala. Para esto, utilizaremos el minMaxScaler del Spark ML para transformar las variables de entrada para que todas tengan valores entre el 0 y el 1.

In [0]:
from pyspark.ml.feature import MinMaxScaler

# Definir las variables de entrada y salida
inputcol = "features"
output = "std_features"

# Definir el MinMaxScaler
minmaxscaler = MinMaxScaler(min=0.0, max=1.0, inputCol=inputcol, outputCol=output)

# Aplicar el MinMaxScaler
results = minmaxscaler.fit(features).transform(features)

In [0]:
from pyspark.ml.feature import VarianceThresholdSelector

# Definir las variables de entrada y salida
inputcol = "std_features"
output = "selectedFeatures"

# Inicializar el VarianceThresholdSelector
selector = VarianceThresholdSelector(varianceThreshold=0.01, featuresCol=inputcol, outputCol=output)

# Aplicar el VarianceThresholdSelector
result = selector.fit(results).transform(results)

# Validar la transformación
prev_num_feat = result.schema["features"].metadata["ml_attr"]["num_attrs"]
new_num_feat = result.schema["selectedFeatures"].metadata["ml_attr"]["num_attrs"]
print(f"El número total de variables de entrada pasó a ser de {prev_num_feat} a {new_num_feat}")

El número total de variables de entrada pasó a ser de 59 a 39


## 2. Ejemplo con variables de entrada homogéneas - Clasificación del comportamiento de agrupamiento de los animales

El comportamiento de agrupamiento es un comportamiento colectivo exibido por los animales que se agrupan hacía un punto específico o mientras migran en una dirección. Desde el punto de vista de modelación matemática, se trata de un comportamiento emergente que surge de reglas simples que son seguidas por los individuos y que no involucra ninguna coordinación central.

Este tipo de comportamiento puede ser simulado en un programa de vida artificial denominado **boids**. Un boid hace referencia a un agente que está permitido moverse de acuerdo a una serie de reglas básicas.

En este ejemplo, procesaremos un conjunto de datos de elevada dimensionalidad para clasificar la existencia o no de un comportamiento de agrupamiento, ya sea como `flocking` (en bandada), `grouped` (agrupados) o `aligned` (alineados).

### 2.1. Sobre los datos

El conjunto de datos (en .csv) contiene las variables de entrada (como valores de campos vectoriales) que nos permitirían predecir si el comportamiento emergente es de agrupamiento o no. El comportamiento de agrupamiento puede ser detectado cuando se presentan algunos de los siguientes aspectos durante su movimiento:
* **En bandada**: se refiere a la forma en que los grupos de aves, insectos, peces u otros animales se acercan unos a otros. Pueden moverse en grupo con la misma velocidad, pero sin toparse entre sí.

![En bandada](assets/1.png)

* **Alineados**: se refiere a sus movimientos en la misma dirección.

![Alineados](assets/2.png)

* **Agrupados**: se refiere a la manera en que se agruparon.

![Agrupados](assets/3.png)

In [0]:
# Leer los datos
df = spark\
     .read\
     .option("header", "true")\
     .option("inferSchema", True)\
     .csv("/FileStore/tables/Swarm_Behaviour.csv")\

spark.sparkContext.setLogLevel("ERROR")

# Reducir el número de Shuffle partitions para 5
spark.conf.set("spark.sql.shuffle.partitions", "5")

# Imprimir el shape
print(f"Este dataset tiene {len(df.columns)} columnas y {df.count()} observaciones")

# Ver los primeros registros
df.limit(5).toPandas()

Este dataset tiene 2401 columnas y 23309 observaciones


x1 
 y1 
 xVel1 
 yVel1 
 xA1 
 yA1 
 xS1 
 yS1 
 xC1 
 yC1 
 nAC1 
 nS1 
 x2 
 y2 
 xVel2 
 yVel2 
 xA2 
 yA2 
 xS2 
 yS2 
 xC2 
 yC2 
 nAC2 
 nS2 
 x3 
 y3 
 xVel3 
 yVel3 
 xA3 
 yA3 
 xS3 
 yS3 
 xC3 
 yC3 
 nAC3 
 nS3 
 x4 
 y4 
 xVel4 
 yVel4 
 xA4 
 yA4 
 xS4 
 yS4 
 xC4 
 yC4 
 nAC4 
 nS4 
 x5 
 y5 
 xVel5 
 yVel5 
 xA5 
 yA5 
 xS5 
 yS5 
 xC5 
 yC5 
 nAC5 
 nS5 
 x6 
 y6 
 xVel6 
 yVel6 
 xA6 
 yA6 
 xS6 
 yS6 
 xC6 
 yC6 
 nAC6 
 nS6 
 x7 
 y7 
 xVel7 
 yVel7 
 xA7 
 yA7 
 xS7 
 yS7 
 xC7 
 yC7 
 nAC7 
 nS7 
 x8 
 y8 
 xVel8 
 yVel8 
 xA8 
 yA8 
 xS8 
 yS8 
 xC8 
 yC8 
 nAC8 
 nS8 
 x9 
 y9 
 xVel9 
 yVel9 
 xA9 
 yA9 
 xS9 
 yS9 
 xC9 
 yC9 
 nAC9 
 nS9 
 x10 
 y10 
 xVel10 
 yVel10 
 xA10 
 yA10 
 xS10 
 yS10 
 xC10 
 yC10 
 nAC10 
 nS10 
 x11 
 y11 
 xVel11 
 yVel11 
 xA11 
 yA11 
 xS11 
 yS11 
 xC11 
 yC11 
 nAC11 
 nS11 
 x12 
 y12 
 xVel12 
 yVel12 
 xA12 
 yA12 
 xS12 
 yS12 
 xC12 
 yC12 
 nAC12 
 nS12 
 x13 
 y13 
 xVel13 
 yVel13 
 xA13 
 yA13 
 xS13 
 yS13 
 xC13 
 yC13 
 nAC13 
 nS13 
 x14 
 y14 
 xVel14 
 yVel14 
 xA14 
 yA14 
 xS14 
 yS14 
 xC14 
 yC14 
 nAC14 
 nS14 
 x15 
 y15 
 xVel15 
 yVel15 
 xA15 
 yA15 
 xS15 
 yS15 
 xC15 
 yC15 
 nAC15 
 nS15 
 x16 
 y16 
 xVel16 
 yVel16 
 xA16 
 yA16 
 xS16 
 yS16 
 xC16 
 yC16 
 nAC16 
 nS16 
 x17 
 y17 
 xVel17 
 yVel17 
 xA17 
 yA17 
 xS17 
 yS17 
 xC17 
 yC17 
 nAC17 
 nS17 
 x18 
 y18 
 xVel18 
 yVel18 
 xA18 
 yA18 
 xS18 
 yS18 
 xC18 
 yC18 
 nAC18 
 nS18 
 x19 
 y19 
 xVel19 
 yVel19 
 xA19 
 yA19 
 xS19 
 yS19 
 xC19 
 yC19 
 nAC19 
 nS19 
 x20 
 y20 
 xVel20 
 yVel20 
 xA20 
 yA20 
 xS20 
 yS20 
 xC20 
 yC20 
 nAC20 
 nS20 
 x21 
 y21 
 xVel21 
 yVel21 
 xA21 
 yA21 
 xS21 
 yS21 
 xC21 
 yC21 
 nAC21 
 nS21 
 x22 
 y22 
 xVel22 
 yVel22 
 xA22 
 yA22 
 xS22 
 yS22 
 xC22 
 yC22 
 nAC22 
 nS22 
 x23 
 y23 
 xVel23 
 yVel23 
 xA23 
 yA23 
 xS23 
 yS23 
 xC23 
 yC23 
 nAC23 
 nS23 
 x24 
 y24 
 xVel24 
 yVel24 
 xA24 
 yA24 
 xS24 
 yS24 
 xC24 
 yC24 
 nAC24 
 nS24 
 x25 
 y25 
 xVel25 
 yVel25 
 xA25 
 yA25 
 xS25 
 yS25 
 xC25 
 yC25 
 nAC25 
 nS25 
 x26 
 y26 
 xVel26 
 yVel26 
 xA26 
 yA26 
 xS26 
 yS26 
 xC26 
 yC26 
 nAC26 
 nS26 
 x27 
 y27 
 xVel27 
 yVel27 
 xA27 
 yA27 
 xS27 
 yS27 
 xC27 
 yC27 
 nAC27 
 nS27 
 x28 
 y28 
 xVel28 
 yVel28 
 xA28 
 yA28 
 xS28 
 yS28 
 xC28 
 yC28 
 nAC28 
 nS28 
 x29 
 y29 
 xVel29 
 yVel29 
 xA29 
 yA29 
 xS29 
 yS29 
 xC29 
 yC29 
 nAC29 
 nS29 
 x30 
 y30 
 xVel30 
 yVel30 
 xA30 
 yA30 
 xS30 
 yS30 
 xC30 
 yC30 
 nAC30 
 nS30 
 x31 
 y31 
 xVel31 
 yVel31 
 xA31 
 yA31 
 xS31 
 yS31 
 xC31 
 yC31 
 nAC31 
 nS31 
 x32 
 y32 
 xVel32 
 yVel32 
 xA32 
 yA32 
 xS32 
 yS32 
 xC32 
 yC32 
 nAC32 
 nS32 
 x33 
 y33 
 xVel33 
 yVel33 
 xA33 
 yA33 
 xS33 
 yS33 
 xC33 
 yC33 
 nAC33 
 nS33 
 x34 
 y34 
 xVel34 
 yVel34 
 xA34 
 yA34 
 xS34 
 yS34 
 xC34 
 yC34 
 nAC34 
 nS34 
 x35 
 y35 
 xVel35 
 yVel35 
 xA35 
 yA35 
 xS35 
 yS35 
 xC35 
 yC35 
 nAC35 
 nS35 
 x36 
 y36 
 xVel36 
 yVel36 
 xA36 
 yA36 
 xS36 
 yS36 
 xC36 
 yC36 
 nAC36 
 nS36 
 x37 
 y37 
 xVel37 
 yVel37 
 xA37 
 yA37 
 xS37 
 yS37 
 xC37 
 yC37 
 nAC37 
 nS37 
 x38 
 y38 
 xVel38 
 yVel38 
 xA38 
 yA38 
 xS38 
 yS38 
 xC38 
 yC38 
 nAC38 
 nS38 
 x39 
 y39 
 xVel39 
 yVel39 
 xA39 
 yA39 
 xS39 
 yS39 
 xC39 
 yC39 
 nAC39 
 nS39 
 x40 
 y40 
 xVel40 
 yVel40 
 xA40 
 yA40 
 xS40 
 yS40 
 xC40 
 yC40 
 nAC40 
 nS40 
 x41 
 y41 
 xVel41 
 yVel41 
 xA41 
 yA41 
 xS41 
 yS41 
 xC41 
 yC41 
 nAC41 
 nS41 
 x42 
 y42 
 xVel42 
 yVel42 
 xA42 
 yA42 
 xS42 
 yS42 
 xC42 
 yC42 
 nAC42 
 nS42 
 x43 
 y43 
 xVel43 
 yVel43 
 xA43 
 yA43 
 xS43 
 yS43 
 xC43 
 yC43 
 nAC43 
 nS43 
 x44 
 y44 
 xVel44 
 yVel44 
 xA44 
 yA44 
 xS44 
 yS44 
 xC44 
 yC44 
 nAC44 
 nS44 
 x45 
 y45 
 xVel45 
 yVel45 
 xA45 
 yA45 
 xS45 
 yS45 
 xC45 
 yC45 
 nAC45 
 nS45 
 x46 
 y46 
 xVel46 
 yVel46 
 xA46 
 yA46 
 xS46 
 yS46 
 xC46 
 yC46 
 nAC46 
 nS46 
 x47 
 y47 
 xVel47 
 yVel47 
 xA47 
 yA47 
 xS47 
 yS47 
 xC47 
 yC47 
 nAC47 
 nS47 
 x48 
 y48 
 xVel48 

Los atributos son **xm**, **ym** como la posición (X,Y) de cada boid, **xVeln** y **yVeln** como el vector de velocidad, **xAm**, **yAm** como el vector de alineación, **xSm**, **ySm** como el vector de separación, **xCm**, **yCm** como el vector de cohesión, **nACm** como el número de cuerpos en el radio de Alineación/Cohesión, y **nSm** como el número de cuerpos en el radio de Separación. Estos atributos se repiten para todos los *m* boids, donde m=1,…,200.

Además, las clases de la variable de salida son binarias, en las que **1** se refiere a "en bandada, agrupado y alineado", y **0** se refiere a "no en bandada, no alineado y no agrupado".

### 2.2 Exploración de los datos

#### 2.2.1. Evaluación de valores nulos
Se evaluó la existencia o no de valores nulos. Como se trata de un conjunto de datos de muchas dimensiones, la búsqueda por valores nulos puede resultar computacionalmente costoso.

In [0]:
%%time

# Determinar la cantidad de valores nulos
def count_missings(spark_df,sort=True):
    """
    Cuenta la cantidad de nulls y nans en cada columna
    """
    nulls = df.select([f.count(f.when(f.isnan(c), c)).alias(c) for c in df.columns]).toPandas()
    if len(nulls) == 0:
        print("¡No hay valores faltantes!")
        return None
    if sort:
        return nulls.rename(index={0: 'count'}).T.sort_values("count",ascending=False)
    return nulls

count_missings(df)

CPU times: user 3.53 s, sys: 1.46 s, total: 5 s
Wall time: 1min 50s


,count
x1,0
nS133,0
y134,0
xVel134,0
yVel134,0
...,...
yC67,0
nAC67,0
nS67,0
x68,0


### 2.3. Transformación mediante VectorAssembler

In [0]:
from pyspark.ml.feature import VectorAssembler

# Definir las variables de entrada y de salida
inputcols = df.drop("Swarm_Behaviour").columns
outputcol = "features"

# Aplicar el VectorAssembler
assembler = VectorAssembler(inputCols = inputcols, outputCol=outputcol)
features = assembler.transform(df)

# Renombrar la variable de salida
features = features.withColumnRenamed("Swarm_Behaviour", "label")

# Eliminar las otras columnas
features = features.select("label","features")
    
# Visualizar el resultado
features.limit(5).toPandas()

,label,features
0,0.0,"(562.05, -0.62, -10.7, -4.33, 0.0, 0.0, 0.0, 0..."
1,0.0,"(175.66, -57.09, 2.31, -2.67, 0.0, 0.0, 0.0, 0..."
2,0.0,"(200.16, -320.07, 4.01, -6.37, 0.0, 0.0, 0.0, ..."
3,0.0,"(316.99, -906.84, 0.85, 9.17, -0.17, 1.03, 0.0..."
4,0.0,"(1277.68, 908.54, -2.02, 8.23, -1.0, 1.0, 0.0,..."


### 2.4. Reducción de la dimensionalidad con PCA

El Principal Component Analysis es un procedimiento estadístico que nos permite representar la información como índices que pueden ser facilmente visualizados y analizados. El uso principal del PCA es reducir la dimensionalidad de las variables de entrada, al mismo tiempo que descubre relaciones entre las observaciones y variables, o entre las variables.

La idea es representar el conjunto de datos en un espacio de **k** dimensiones. Para hacerlo, el PCA:
* Estandariza el rango de variables iniciales continuas.
* Calcula la matriz de covarianza para identificar correlaciones.
* Calcula los vectores propios y los valores propios de la matriz de covarianza para identificar los componentes principales.
* Crea un vector de características para decidir qué componentes principales mantener.
* Reformula los datos a lo largo de los ejes de los componentes principales.

**Nota**: los datos necesitan estar en una misma escala para aplicar el PCA.

In [0]:
# Definir las variables de entrada y salida
inputcol = "features"
output = "std_features"

# Definir el MinMaxScaler
minmaxscaler = MinMaxScaler(min=0.0, max=1.0, inputCol=inputcol, outputCol=output)

# Aplicar el MinMaxScaler
features = minmaxscaler.fit(features).transform(features)

Finalmente, aplicaremos el PCA sobre los datos de entrada. Definiremos el valor de **k** como 400.

In [0]:
from pyspark.ml.feature import PCA

# Iniciar el transformador
pca = PCA(k=400, inputCol="std_features", outputCol="pcaFeatures")
model = pca.fit(features)

# Aplicar el PCA
result = model.transform(features)

# # Validar la transformación
prev_num_feat = result.schema["features"].metadata["ml_attr"]["num_attrs"]
new_num_feat = result.schema["pcaFeatures"].metadata["ml_attr"]["num_attrs"]
print(f"El número total de variables de entrada pasó a ser de {prev_num_feat} a {new_num_feat}")

El número total de variables de entrada pasó a ser de 2400 a 400


No vamos a entrar en detalles sobre el valor de **k**, apenas necesitamos saber que este valor representa la cantidad de componentes principales. Es posible aproximar un valor ideal al analizar y comparar la sumatoria de las varianzas explicadas de los componentes principales (*accumulated explained variance*) para cada nivel arbitrario de **k**. Para obtener las varianzas de todos los componentes, se utiliza el método `explainedVariance` del estimador:

In [0]:
model.explainedVariance

Out[18]: DenseVector([0.2282, 0.1088, 0.0523, 0.0259, 0.0163, 0.0141, 0.0113, 0.0108, 0.0099, 0.0091, 0.0089, 0.0084, 0.0076, 0.0076, 0.0065, 0.0064, 0.006, 0.005, 0.0048, 0.0046, 0.0044, 0.0041, 0.0039, 0.0038, 0.0037, 0.0036, 0.0035, 0.0035, 0.0035, 0.0034, 0.0034, 0.0033, 0.0033, 0.0032, 0.0032, 0.0031, 0.0031, 0.003, 0.003, 0.0029, 0.0029, 0.0029, 0.0029, 0.0028, 0.0028, 0.0028, 0.0027, 0.0027, 0.0027, 0.0026, 0.0026, 0.0025, 0.0025, 0.0025, 0.0025, 0.0025, 0.0024, 0.0024, 0.0024, 0.0023, 0.0023, 0.0023, 0.0023, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0021, 0.0021, 0.0021, 0.0021, 0.002, 0.002, 0.002, 0.002, 0.002, 0.0019, 0.0019, 0.0019, 0.0019, 0.0019, 0.0019, 0.0018, 0.0018, 0.0018, 0.0018, 0.0018, 0.0018, 0.0017, 0.0017, 0.0017, 0.0017, 0.0017, 0.0017, 0.0017, 0.0017, 0.0016, 0.0016, 0.0016, 0.0016, 0.0016, 0.0016, 0.0015, 0.0015, 0.0015, 0.0015, 0.0015, 0.0015, 0.0015, 0.0014, 0.0014, 0.0014, 0.0014, 0.0014, 0.0014, 0.0014, 0.0014, 0.0014, 0.0013, 0.0013, 0.0013, 0.0013, 0.0

### Conclusiones

Hemos demostrado algunos de los pasos para la selección de características y reducción de dimensionalidad con los módulos del Spark ML. Se utilizó para un ejemplo las dos herramientas principales para la selección de características por método de filtro: **UnivariateFeatureSelector y el VarianceThresholdSelector**; mientras que en un otro ejemplo, hemos logrado reducir miles de variables a 400 gracias al **Principal Component Analysis**.

### Bibliografía


* Artículo científico sobre los desafíos de la selección de características en Big Data: https://ieeexplore.ieee.org/document/8624280
* Artículo científico analizando las técnicas de selección de características propuestas en la literatura sobre datos de streaming: https://www.emerald.com/insight/content/doi/10.1016/j.aci.2019.01.001/full/html
* Explicación didactica sobre el Principal Component Analysis (PCA) en *profesordata.com*: https://profesordata.com/2020/09/01/reduccion-de-la-dimensionalidad-analisis-de-componentes-principales-pca/
* Documentación sobre el UnivariateFeatureSelector: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.UnivariateFeatureSelector.html
* Documentación sobre el VarianceThresholdSelector: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.VarianceThresholdSelector.html